In [1]:
"""
Connect senior:Mainland
"""
import pandas as pd
import json
import requests
def ssql(sql):
    admin_token = '2cebeb5c7edac5ef0d7c5a640e69fc7d43bf2cd6a24ce43dfee3dd33662c0bab'
    url = 'http://sensor.wb-intra.com/api/sql/query?token=%s&project=production' % admin_token
    data = {'q': sql, 'format': 'json'}
    req = requests.post(url,data)
    req_dec = req.content.decode()
    try:
        req_json = json.loads('[' + req_dec.replace('\n', ',')[:-1] + ']')
        df_d_id = pd.DataFrame(req_json)
        return df_d_id
    except:
        print(req_dec)

In [2]:
df = ssql("""
SELECT t2.*
FROM
  (SELECT date,distinct_id
   FROM events
   WHERE event = 'login'
     AND appId IN ('20014',
                   '30015')
     AND date BETWEEN '2020-11-20' AND current_date() - interval 1 DAY
   GROUP BY 1,
            2)t1
JOIN
  (SELECT date,distinct_id,
               count(distinct_id)AS finish_num
   FROM events
   WHERE event = 'mission'
     AND SOURCE = 'killer_dailyTask'
     AND missionOp = 'finish'
     AND date BETWEEN '2020-11-20' AND current_date() - interval 1 DAY
   GROUP BY 1,
            2)t2 ON t1.date = t2.date
AND t1.distinct_id = t2.distinct_id
""")

In [3]:
df

,date,distinct_id,finish_num
0,2020-11-25,142865581,9
1,2020-11-24,111452454,4
2,2020-11-25,116622793,6
3,2020-11-24,114656869,1
4,2020-11-25,117377518,13
...,...,...,...
590287,2020-11-22,126660102,5
590288,2020-11-25,144923032,4
590289,2020-11-24,117951860,2
590290,2020-11-21,114509963,6


In [4]:
df_copy = df.copy()

In [14]:
df_copy.groupby(['date']).describe()['finish_num']

,count,mean,std,min,25%,50%,75%,max
date,,,,,,,,
2020-11-20,33292.0,3.096780,2.049918,1.0,1.0,3.0,5.0,10.0
2020-11-21,125267.0,4.105606,2.563089,1.0,2.0,4.0,6.0,19.0
2020-11-22,140284.0,4.137528,2.603152,1.0,2.0,4.0,6.0,22.0
2020-11-23,73793.0,3.737888,2.501783,1.0,2.0,3.0,6.0,17.0
2020-11-24,71769.0,3.699968,2.494299,1.0,2.0,3.0,6.0,17.0
2020-11-25,72773.0,3.657098,2.503977,1.0,1.0,3.0,5.0,17.0
2020-11-26,73114.0,3.679842,2.514498,1.0,1.0,3.0,5.0,17.0


In [19]:
df_copy.groupby(['date'])['finish_num'].describe().to_clipboard()

In [2]:
df_ad = ssql("""

SELECT date,distinct_id,
            count(distinct_id)AS play_amount
FROM events
WHERE event = 'behavior'
  AND SOURCE = 'video_adv'
  AND gameTypeId = 1800
  AND date BETWEEN '2020-11-20' AND current_date() - interval 1 DAY
GROUP BY 1,
         2
""")

In [3]:
df_ad

,date,play_amount,distinct_id
0,2020-11-29,1,98231408
1,2020-11-23,8,143483150
2,2020-11-22,5,133452458
3,2020-11-28,1,117291042
4,2020-11-23,6,128840189
...,...,...,...
27556,2020-11-22,1,122836119
27557,2020-11-28,1,37944222
27558,2020-11-25,5,142197700
27559,2020-11-30,7,115788573


In [4]:
df_ad.groupby(['date'])['play_amount'].describe().to_clipboard()

In [5]:
df_ad = df_ad[df_ad['play_amount']<=9]
df_ad

,date,play_amount,distinct_id
0,2020-11-29,1,98231408
1,2020-11-23,8,143483150
2,2020-11-22,5,133452458
3,2020-11-28,1,117291042
4,2020-11-23,6,128840189
...,...,...,...
27556,2020-11-22,1,122836119
27557,2020-11-28,1,37944222
27558,2020-11-25,5,142197700
27559,2020-11-30,7,115788573


In [6]:
df_ad.groupby(['date'])['play_amount'].describe().to_clipboard()

In [4]:
df_dur = ssql("""
SELECT t2.*
FROM
  (SELECT date,distinct_id
   FROM events
   WHERE event = 'behavior'
     AND SOURCE = 'video_adv'
     AND gameTypeId = 1800
     AND date BETWEEN '2020-11-20' AND current_date() - interval 1 DAY
   GROUP BY 1,
            2)t1
JOIN
  (SELECT date, distinct_id,
                gameSubtype,
                sum(duration/60000)AS MINUTE
   FROM events
   WHERE event = 'gameOver'
     AND gameTypeId = 1800
     AND date BETWEEN '2020-11-20' AND current_date()- interval 1 DAY
   GROUP BY 1,
            2,
            3)t2 ON t1.date = t2.date
AND t1.distinct_id =t2.distinct_id
""")

In [5]:
df_dur

,date,distinct_id,gamesubtype,minute
0,2020-11-25,142865581,3,707.443
1,2020-11-25,142865581,1,136.963
2,2020-11-22,112736864,1,20.053
3,2020-11-22,112736864,3,12.314
4,2020-11-22,112736864,2,22.099
...,...,...,...,...
49255,2020-11-25,75366335,2,3.524
49256,2020-11-28,145211984,3,9.370
49257,2020-11-28,145211984,1,105.679
49258,2020-11-23,116217653,1,4.915


In [15]:
df_dur.groupby(['date','gamesubtype'])['minute'].describe().to_clipboard()

In [17]:
df_dur_1 = ssql("""
SELECT t1.*
FROM
  (SELECT date, distinct_id,
                gameSubtype,
                sum(duration/60000)AS MINUTE
   FROM events
   WHERE event = 'gameOver'
     AND gameTypeId = 1800
     AND date BETWEEN '2020-11-20' AND current_date()- interval 1 DAY
   GROUP BY 1,
            2,
            3)t1
LEFT JOIN
  (SELECT date,distinct_id
   FROM events
   WHERE event = 'behavior'
     AND SOURCE = 'video_adv'
     AND gameTypeId = 1800
     AND date BETWEEN '2020-11-20' AND current_date() - interval 1 DAY
   GROUP BY 1,
            2)t2 ON t1.distinct_id = t2.distinct_id
AND t1.date = t2.date
WHERE t2.distinct_id IS NULL
""")

In [18]:
df_dur_1

,date,distinct_id,gamesubtype,minute
0,2020-11-22,118623018,1,9.873
1,2020-11-21,129735675,1,5.594
2,2020-11-25,116622793,1,23.802
3,2020-11-25,116622793,3,71.703
4,2020-11-25,140773186,2,16.595
...,...,...,...,...
4571327,2020-11-27,90186707,1,4.717
4571328,2020-11-26,111807916,3,48.112
4571329,2020-11-20,107803982,2,27.702
4571330,2020-11-20,107803982,3,122.834


In [19]:
df_dur_1.groupby(['date','gamesubtype'])['minute'].describe().to_clipboard()

In [5]:
df_gameNum = ssql("""
SELECT t1.*
FROM
  (SELECT date, distinct_id,
                gameSubtype,
                count(distinct_id)AS game_num
   FROM events
   WHERE event = 'gameOver'
     AND gameTypeId = 1800
     AND date BETWEEN '2020-11-20' AND current_date()- interval 1 DAY
   GROUP BY 1,
            2,
            3)t1
LEFT JOIN
  (SELECT date,distinct_id
   FROM events
   WHERE event = 'behavior'
     AND SOURCE = 'video_adv'
     AND gameTypeId = 1800
     AND date BETWEEN '2020-11-20' AND current_date() - interval 1 DAY
   GROUP BY 1,
            2)t2 ON t1.distinct_id = t2.distinct_id
AND t1.date = t2.date
WHERE t2.distinct_id IS NULL
""")

In [4]:
df_gameNum.groupby(['date','gamesubtype'])['game_num'].describe().to_clipboard()

In [6]:
df_gameNum

,date,distinct_id,gamesubtype,game_num
0,2020-11-22,118623018,1,1
1,2020-11-21,129735675,1,1
2,2020-11-25,116622793,1,6
3,2020-11-25,116622793,3,8
4,2020-11-25,140773186,2,6
...,...,...,...,...
4571327,2020-11-29,113195601,1,13
4571328,2020-11-29,113195601,2,1
4571329,2020-11-20,107803982,2,8
4571330,2020-11-20,107803982,3,19


In [7]:
df_gameNum.groupby(['date','gamesubtype'])['game_num'].describe().to_clipboard()